In [20]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'spark-3.4.0-bin-hadoop3.tgz'


In [21]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [22]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
homes_df.head()

23/12/09 12:44:12 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date='2022-04-08', date_built='2016', price='936923', bedrooms='4', bathrooms='3', sqft_living='3167', sqft_lot='11733', floors='2', waterfront='1', view='76')

In [23]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('home_sales')


In [26]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result1 = homes_df.filter(col("bedrooms").cast("int") == 4)\
                 .groupBy("date")\
                 .agg(round(avg("price"), 2).alias("avg_price"))\
                 .orderBy("date")

result1.show()

+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-01-01| 365088.0|
|2019-01-02|271644.29|
|2019-01-03|381218.86|
|2019-01-04|359586.14|
|2019-01-05|323683.57|
|2019-01-06|301091.13|
|2019-01-07| 327474.5|
|2019-01-08|304567.83|
|2019-01-09| 243619.0|
|2019-01-10| 287844.4|
|2019-01-11|277574.29|
|2019-01-12| 218507.0|
|2019-01-13|294603.86|
|2019-01-14|284434.57|
|2019-01-15| 254288.5|
|2019-01-16| 335106.5|
|2019-01-17|333439.09|
|2019-01-18|297542.56|
|2019-01-19|294484.22|
|2019-01-20|312395.33|
+----------+---------+
only showing top 20 rows



In [24]:
print(homes_df.columns)


['id', 'date', 'date_built', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view']


In [27]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result2 = homes_df.filter((col("bedrooms").cast("int") == 3) & (col("bathrooms").cast("int") == 3))\
                 .groupBy("date")\
                 .agg(round(avg("price"), 2).alias("avg_price"))\
                 .orderBy("date")

result2.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-01-01| 316073.5|
|2019-01-02|344639.38|
|2019-01-03| 124107.0|
|2019-01-04| 209626.0|
|2019-01-05| 219228.8|
|2019-01-06|359786.75|
|2019-01-07| 347663.0|
|2019-01-08| 234974.0|
|2019-01-09| 393417.8|
|2019-01-10|232471.67|
|2019-01-11|356400.33|
|2019-01-12| 274625.0|
|2019-01-13| 263419.0|
|2019-01-14|295011.56|
|2019-01-15|322817.83|
|2019-01-16| 295334.6|
|2019-01-17|295683.67|
|2019-01-18| 258211.2|
|2019-01-19|225870.83|
|2019-01-20| 252820.0|
+----------+---------+
only showing top 20 rows



In [28]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result3 = homes_df.filter((col("bedrooms").cast("int") == 3) &
                          (col("bathrooms").cast("int") == 3) &
                          (col("floors") == 2) &
                          (col("sqft_living").cast("int") >= 2000))\
                 .groupBy("date")\
                 .agg(round(avg("price"), 2).alias("avg_price"))\
                 .orderBy("date")

result3.show()



+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-01-01| 316073.5|
|2019-01-04| 209626.0|
|2019-01-05| 134866.0|
|2019-01-06|341909.33|
|2019-01-07| 372195.0|
|2019-01-09| 431916.0|
|2019-01-11| 170073.0|
|2019-01-13| 309647.0|
|2019-01-14| 434033.0|
|2019-01-15| 432945.0|
|2019-01-16| 194432.5|
|2019-01-17|302354.33|
|2019-01-18| 306050.0|
|2019-01-19| 214817.0|
|2019-01-20| 193427.0|
|2019-01-21| 232836.5|
|2019-01-22| 159979.0|
|2019-01-28| 346279.0|
|2019-01-31| 277476.0|
|2019-02-02| 447509.0|
+----------+---------+
only showing top 20 rows



In [29]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 5.125999450683594e-05 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')
print("Is home_sales cached?", spark.catalog.isCached("home_sales"))

Is home_sales cached? True


In [32]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
result3_cached = spark.sql("SELECT view, round(avg(price), 2) as avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view")
result3_cached.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 3.3855438232421875e-05 seconds ---


In [33]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
homes_df.write.partitionBy("date_built").parquet("home_sales_partitioned_parquet")

In [35]:
# 11. Read the parquet formatted data.
partitioned_parquet_df = spark.read.parquet("home_sales_partitioned_parquet")
print(partitioned_parquet_df)

DataFrame[id: string, date: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, date_built: int]


In [36]:
# 12. Create a temporary table for the parquet data.
partitioned_parquet_df = spark.read.parquet("home_sales_partitioned_parquet")
partitioned_parquet_df.createOrReplaceTempView("home_sales_partitioned_parquet")

In [41]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.
# Run the query on the parquet data
# Run the query on the parquet data
start_time_parquet = time.time()

result_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_parquet
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")

end_time_parquet = time.time()
runtime_parquet = end_time_parquet - start_time_parquet

# Show the result and print the runtime
result_parquet.show()
print(f"Runtime for Parquet query: {round(runtime_parquet, 2)} seconds")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales_parquet` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Sort ['view ASC NULLS FIRST], true
+- 'Aggregate ['view], ['view, 'ROUND('AVG('price), 2) AS avg_price#989]
   +- 'Filter ('price >= 350000)
      +- 'UnresolvedRelation [home_sales_parquet], [], false


In [37]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [38]:
# 15. Check if the home_sales is no longer cached
print("Is home_sales cached?", spark.catalog.isCached("home_sales"))



Is home_sales cached? False
